# Finding ground states with DMRG

In [1]:
using MPStates, LinearAlgebra, Test

Let's first define an `Mps` and create a Hubbard Hamiltonian like
$$ H = 0.5\sum^{L-1}_{i=1} c^\dagger_i c_{i+1} + 0.7\sum^{L-1}_{i=1} n_i n_{i+1} $$

In [2]:
# Number of sites.
L = 10
# Type of the Mps.
T = Float64
# Physical dimension.
d = 2

psi = Mps(Float64, L, "W");

# Matrices that describe the Hamiltonian.
# Hopping matrix.
J = 0.5*Symmetric(diagm(1 => ones(L-1)))
# Interaction matrix.
V = 0.7*diagm(1 => ones(L-1))

# Build the Hamiltonian.
H = Mpo(T, L, d)
add_ops!(H, "c+", "c", J, ferm_op="Z")
add_ops!(H, "n", "n", V);

# DMRG minimization algorithm

This is done with the `dmrg!` function. It accepts as input an `Mps`, the objective `Mpo`, and a set of options for the DMRG algorithm. These options are a structure `DMRGOpts` with fields:
* `algorithm::String`: DMRG algorithm used for minimizing the state. Can be:
  - `"DMRG1"` for one site DMRG.
  - `"DMRG2"` for two site DMRG.
  - `"DMRG3S"` for strictly single site DMRG with subspace expansion, an improved version of the one site DMRG algorithm that doesn't get trapped in local minima (https://arxiv.org/abs/1501.05504).
* `nsweeps::Int`: number of sweeps.
* `maxm::Vector{Int}`: maximum bond dimension at every sweep.
* `cutoff::Vector{Float64}`: cutoff in SVD factorization at every sweep.
* `dmrg_tol::Float64`: stop the algorithm when the change in the variance of the
        state is less than dmrg_tol.
* `show_trace::Int`: output information at every step of the minmization:
        - 0: no info given.
        - 1: energy, variance and their variation with respect to their last
            value after every right + left sweep.
        - 2: energy and size of the local Hamiltonian at every step of every
            sweep.
* `lanczos_iters::Int`: number of maximum iterations in Arpack solver for
        minimizing local Hamiltonians.
* `α::Float64`: noise term in DMRG3S.

In [3]:
# Maximum bond dimension per sweep.
maxm = [10, 20, 40, 40, 40, 40]
# Cutoff in SVD factorization per sweep.
cutoff = [1e-5, 1e-5, 1e-6, 1e-8, 1e-8, 1e-8]
dmrg_opts = DMRGOpts("DMRG3S", maxm, cutoff)

# Alternatively, if you only want one cutoff value, e.g. 1e-6, you could start the options as:
dmrg_opts = DMRGOpts("DMRG3S", maxm, 1e-6)

# Now let's find the ground state.
E, var = dmrg!(psi, H, dmrg_opts)

println("The energy after each sweep was: $E")

Done sweep 1, max bond dimension: 8
    E: -2.581036e+00, ΔE: -1.00e+05
    var: 5.102695e-03, Δvar: -1.00e+05
    Elapsed time: 3.967707433 s
Done sweep 2, max bond dimension: 8
    E: -2.586807e+00, ΔE: -5.77e-03
    var: 2.445423e-03, Δvar: -2.66e-03
    Elapsed time: 0.018160865 s
Done sweep 3, max bond dimension: 8
    E: -2.586809e+00, ΔE: -2.06e-06
    var: 2.453811e-03, Δvar: 8.39e-06
    Elapsed time: 0.024918495 s
Done sweep 4, max bond dimension: 8
    E: -2.586809e+00, ΔE: -8.10e-09
    var: 2.453189e-03, Δvar: -6.21e-07
    Elapsed time: 0.019006878 s
The variance of psi has converged. Finished DMRG.
The energy after each sweep was: [-2.5810355896524597, -2.5868068715740478, -2.5868089272859085, -2.586808935386919]
